## Imports

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import nltk
from sklearn.feature_extraction import text
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import numpy as np


nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


c:\Users\fabio\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load Datasets

In [2]:
# Load your data
data = pd.read_json('../data/data.jsonl', lines=True)
test_data = pd.read_json('../data/test_final.jsonl', lines=True)
train_data = pd.read_json('../data/train_final.jsonl', lines=True)
validation_data = pd.read_json('../data/validation_final.jsonl', lines=True)

# Remove duplicates
test_data = test_data.drop_duplicates(subset=['text'])
train_data = train_data.drop_duplicates(subset=['text'])
validation_data = validation_data.drop_duplicates(subset=['text'])


## Pre-processing

In [3]:
my_stop_words = text.ENGLISH_STOP_WORDS
words_to_keep = frozenset(['no', 'couldnt', 'cry', 'not', 'cant', 'cannot', 'nor', 'except', 'nobody',
                           'off', 'but', 'serious', 'enough', 'nothing', 'alone', 'down', 'only', 'without'])
my_stop_words = my_stop_words - words_to_keep

def pre_process_data(dataset):
    # Tokenize
    #dataset['text'] = dataset['text'].apply(word_tokenize)
    # Remove stop words
    dataset['text'] = dataset['text'].apply(lambda x: [word for word in x if word.lower() not in my_stop_words])
    return dataset


train_data = pre_process_data(train_data)
validation_data = pre_process_data(validation_data)
test_data = pre_process_data(test_data)


## Tokenization Using BERT Tokenizer:
BERT tokenizer requires the data to be in a specific format

???????? MAS ESTÁ CERTO ANTES TAMBEM TOKENIZARMOS ????????


* podemos usar o bert-base-uncased e o roberta-based e comparar dps
* maybe usar embeddings tb depois


In [4]:
# Initialize the BERT tokenizer and model
model_name = "bert-base-uncased"

# SUPOSTAMENTE ESTE TB É BOM
# model_name = "roberta-base"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Tokenize using BERT tokenizer
def tokenize_data(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)

train_encodings = tokenize_data(train_data['text'].apply(lambda x: ' '.join(x)).tolist())
val_encodings = tokenize_data(validation_data['text'].apply(lambda x: ' '.join(x)).tolist())
test_encodings = tokenize_data(test_data['text'].apply(lambda x: ' '.join(x)).tolist())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Create Dataset Objects

In [5]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_data['label'].tolist())
val_dataset = TextDataset(val_encodings, validation_data['label'].tolist())
test_dataset = TextDataset(test_encodings, test_data['label'].tolist())

## Training

#### Function to compute the training metrics

In [6]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


#### Training hyppertunning

In [44]:
## depois coloco aqui quando passar o trainer parar de me dar erro ?????

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
 # Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
)

print("Training labels range: ", min(train_data['label']), "to", max(train_data['label']))
print("Validation labels range: ", min(validation_data['label']), "to", max(validation_data['label']))
print("Test labels range: ", min(test_data['label']), "to", max(test_data['label']))

print("Training data NaN values:", train_data.isnull().values.any())
print("Validation data NaN values:", validation_data.isnull().values.any())
print("Test data NaN values:", test_data.isnull().values.any())

 # Initialize Trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print(os.getenv('CUDA_LAUNCH_BLOCKING'))
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print(trainer)

Training labels range:  0 to 5
Validation labels range:  0 to 5
Test labels range:  0 to 5
Training data NaN values: False
Validation data NaN values: False
Test data NaN values: False
1


c:\Users\fabio\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


## Training Evaluation

In [10]:
trainer.train()
eval_results = trainer.evaluate()
print(eval_results)

  6%|▋         | 500/7989 [03:25<1:13:50,  1.69it/s]

{'loss': 1.7972, 'learning_rate': 1.8748278883464764e-05, 'epoch': 0.19}


 13%|█▎        | 1000/7989 [08:20<1:11:11,  1.64it/s]

{'loss': 1.7642, 'learning_rate': 1.749655776692953e-05, 'epoch': 0.38}


 19%|█▉        | 1500/7989 [13:28<1:00:43,  1.78it/s]

...

{'eval_loss': 0.4559192657470703, 'eval_accuracy': 0.856428306780289, 'eval_f1': 0.857196278947809, 'eval_precision': 0.8695140701219711, 'eval_recall': 0.856428306780289, 'eval_runtime': 158.0496, 'eval_samples_per_second': 34.154, 'eval_steps_per_second': 2.139, 'epoch': 3.0}


100%|██████████| 7989/7989 [1:48:29<00:00,  1.23it/s]


{'train_runtime': 6509.7128, 'train_samples_per_second': 19.635, 'train_steps_per_second': 1.227, 'train_loss': 0.8646236930118513, 'epoch': 3.0}


100%|██████████| 338/338 [02:35<00:00,  2.18it/s]

{'eval_loss': 0.4559192657470703, 'eval_accuracy': 0.856428306780289, 'eval_f1': 0.857196278947809, 'eval_precision': 0.8695140701219711, 'eval_recall': 0.856428306780289, 'eval_runtime': 155.5011, 'eval_samples_per_second': 34.714, 'eval_steps_per_second': 2.174, 'epoch': 3.0}
